# Import des outils / jeu de données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import prince
import seaborn as sns
import statsmodels.api as sm
from mlxtend.plotting import plot_pca_correlation_graph
from scipy.stats import bartlett, shapiro
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, StandardScaler
from statsmodels.formula.api import ols

In [ ]:
np.random.seed(0)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "data/data-cleaned-feature-engineering.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

In [ ]:
df_transforme = pd.read_csv(
    "data/data-transformed.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

## Variables globales

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

# Analyse multi-variée

## Clustering

In [ ]:
scaler = RobustScaler()
df_apres_scale = pd.DataFrame(
    scaler.fit_transform(df[var_numeriques]),
    columns=df[var_numeriques].columns,
)

In [ ]:
k3 = KMeans(n_clusters=3)
k4 = KMeans(n_clusters=4)

In [ ]:
k3.fit(df_apres_scale[var_numeriques])
k4.fit(df_apres_scale[var_numeriques])

In [ ]:
df["cluster"] = k3.labels_

In [ ]:
sns.histplot(df["cluster"])

In [ ]:
sns.histplot(df, x="Income", hue="cluster", kde=True)

In [ ]:
sns.scatterplot(df, x="Year_Birth", y="Income", hue="cluster")

In [ ]:
df[var_numeriques + ["cluster"]]

In [ ]:
df["cluster"] = k4.labels_
sns.histplot(df["cluster"])

In [ ]:
sns.histplot(df, x="Income", hue="cluster", kde=True)

In [ ]:
sns.scatterplot(df, x="Year_Birth", y="Income", hue="cluster")

## Analyse en Composantes Principales (ACP)

In [ ]:
# todo: centrer / réduire

In [ ]:
scaler = StandardScaler()

In [ ]:
df_apres_scale = pd.DataFrame(
    scaler.fit_transform(df[var_numeriques]), columns=df[var_numeriques].columns
)

In [ ]:
df_apres_scale

In [ ]:
# for var in var_numeriques:
#     _, ax = plt.subplots(1, 2, figsize=(8, 2))
#     sns.boxplot(df_apres_scale[var], width=0.25, ax=ax[0])
#     sns.histplot(df_apres_scale[var], kde=True, ax=ax[1])
#     plt.show()

In [ ]:
acp = PCA(random_state=0)

In [ ]:
acp.fit(df_apres_scale)

In [ ]:
variance_expliquee = pd.Series(
    acp.explained_variance_ratio_, index=df[var_numeriques].columns
)

In [ ]:
variance_expliquee

In [ ]:
variance_expliquee.plot.barh()

In [ ]:
df_acp = pd.DataFrame(acp.fit_transform(df_apres_scale), index=df.index)

In [ ]:
df_acp.head()

In [ ]:
sns.scatterplot(df_acp, x=0, y=1)

### Cercle de corrélations

In [ ]:
_, correlation_matrix = plot_pca_correlation_graph(
    df_apres_scale,
    df_apres_scale.columns,
    X_pca=df_acp.iloc[:, :2],
    explained_variance=acp.explained_variance_[:2],
    dimensions=(1, 2),
)

In [ ]:
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

### Test clusters

In [ ]:
df_acp["clusterk3"] = k3.labels_
df_acp["clusterk4"] = k4.labels_

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue="clusterk3")

In [ ]:
df_acp.head()

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue=df["Response"])

In [ ]:
sns.scatterplot(df_acp, x=0, y=2, hue=df["Response"])

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue="clusterk4")

## Analyse Factorielle des Correspondances (AFC)

In [ ]:
table_contingence = pd.crosstab(df["Kidhome"], df["Teenhome"])

In [ ]:
table_contingence

In [ ]:
ca = prince.CA(
    # n_components=3,
    # n_iter=3,
    # copy=True,
    # check_input=True,
    # engine='sklearn',
    random_state=0
)

ca = ca.fit(table_contingence)

In [ ]:
ca.eigenvalues_summary

In [ ]:
ca.plot(table_contingence)

In [ ]:
table_contingence = pd.crosstab(df["Marital_Status"], df["Education"])

In [ ]:
table_contingence

In [ ]:
ca = prince.CA(random_state=0)
ca = ca.fit(table_contingence)

In [ ]:
ca.eigenvalues_summary

In [ ]:
ca.plot(table_contingence)

In [ ]:
# todo: à interpréter

## Analyse des Correspondances Multiples (ACM)

In [ ]:
mca = prince.MCA(random_state=0)
mca = mca.fit(df[var_categoriques])

In [ ]:
mca.plot(df[var_categoriques])

In [ ]:
# todo: à interpréter

# ANOVA

## Problématique

Nous allons tester l'indépendance entre la variable `Response` (catégorique, binaire) et `Income` (quantitative continue), pour répondre à la question : "le revenu influence-t-il la réponse aux campagnes marketing ?".

In [ ]:
sns.boxplot(df, x="Response", y="Income")

On cherche à déterminer si les moyennes des groupes sont significativement différentes. On pose donc :

$H_0$ : Les moyennes de chaque groupe sont égales si la p-value $> 5\%$
$H_1$ : Les moyennes de chaque groupe ne sont pas toutes égales si la p-value $< 5\%$


## Hypothèses

1) l’indépendance entre les échantillons de chaque groupe
2) l’égalité des variances que l’on peut verifier avec un test de Bartlett.
3) la normalité des résidus avec un test de Shapiro.

### Indépendance

### Egalité des variances

Voici la variance de chaque groupe :

In [ ]:
df.groupby("Response")["Income"].agg("var")

Nous allons effectuer un test de Bartlett pour vérifier l'égalité des variances.

$H_0$ : Les variances de chaque groupe sont égales si p-value $> 5\%$
$H_1$ : Les variances de chaque groupe ne sont pas toutes égales $< 5\%$


In [ ]:
bartlett(
    df["Income"][df["Response"] == "0"],
    df["Income"][df["Response"] == "1"],
)

Notre p-value est inférieure à 5%, donc les variances ne sont pas toutes égales au (risque de 5%).

### Normalité des résidus

Nous utilisons le test de Shapiro-Wilk pour vérifier la normalité des résidus.

H0 : Les résidus suivent une loi normale si p-value > 5%
H1 : Les résidus ne suivent pas une loi normale si p-value < 5%

In [ ]:
model = ols("Income ~ Response", data=df).fit()
shapiro(model.resid)

Comme la p-value est inférieure à 5%, les résidus ne suivent pas une loi normale (au risque de 5%).

## Test d'ANOVA

H0 : Les moyennes de chaque groupe sont égales si p-value > 5%
H1 : Les moyennes de chaque groupe ne sont pas toutes égales < 5%

In [ ]:
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

Comme aucune hypothèse n'est vérifiée, le test n'est pas fiable.

In [ ]:
# todo: tenter le GLS

# Sauvegarde du Dataframe

In [ ]:
cluster3 = pd.DataFrame(k3.labels_, columns=["cluster3"])
cluster4 = pd.DataFrame(k4.labels_, columns=["cluster4"])

In [ ]:
cluster3.to_csv("data/clusters-kmeans3.csv")
cluster4.to_csv("data/clusters-kmeans4.csv")